In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "vocative"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-10-at-2023-06-02-18-09-d23c0c54.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_vocative_export_big_v2_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[0]

{'id': 71678,
 'annotations': [{'id': 22041,
   'completed_by': 1,
   'result': [{'value': {'start': 22,
      'end': 31,
      'text': 'noor kana',
      'labels': ['vocative_phrases']},
     'id': 'V2PLGnKyFJ',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['redundant comma']},
     'id': 'lwoLvHjx3g',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-06-02T16:45:14.387082Z',
   'updated_at': '2023-06-02T16:45:14.387117Z',
   'lead_time': 9.783,
   'prediction': {'id': 71677,
    'model_version': 'undefined',
    'created_ago': '0\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 22,
       'end': 31,
       'text': ['noor', 'kana'],
       'labels': ['vocative_phrases']}}],
    'score': None,
    'cl

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [9]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,71678,Train/ilu_valton_ud211.conllu,""" Ära ainult valeta , noor kana ! "" ütles vaen...",22,31,noor kana,redundant comma,""" Ära ainult valeta , ! "" ütles vaenulik tädi ...",73.3,80.0,80.0
1,71679,Dev/ilu_orlau_ud211.conllu,""" Kristin ! "" paanilises hirmus raputas Juudas...",0,13,""" Kristin ! """,free,paanilises hirmus raputas Juudas tüdrukut õlga...,100.0,100.0,100.0
2,71680,Train/ilu_vilep_ud211.conllu,""" Noo teate , poisid ! "" ühmas ta omaette .",12,20,", poisid",free,""" Noo teate ! "" ühmas ta omaette .",66.7,66.7,77.8
3,71681,Dev/ilu_orlau_ud211.conllu,""" Juudas ... "" sosistas ta .",0,14,""" Juudas ... """,bound,sosistas ta .,100.0,100.0,100.0
4,71682,Train/ilu_kivirahk_ud211.conllu,""" Sa oled kõigest täpselt aru saanud , õilis r...",37,51,", õilis rüütel",free,""" Sa oled kõigest täpselt aru saanud ! "" kinni...",75.0,75.0,83.3
...,...,...,...,...,...,...,...,...,...,...,...
127,71805,Train/ilu_maailm_ud211.conllu,"Kas sa ka veini tahad , Gerda ? """,22,29,", Gerda",free,"Kas sa ka veini tahad ? """,100.0,100.0,100.0
128,71806,Train/ilu_kivirahk_ud211.conllu,""" Ülekohtune olete , papake , oi , kui ülekoht...",19,27,", papake",free,""" Ülekohtune olete , oi , kui ülekohtune ! "" s...",84.0,88.0,88.0
129,71807,Train/ilu_kivirahk_ud211.conllu,""" Mis maakonnast sa oled , laulupeoline ? "" kü...",25,39,", laulupeoline",free,""" Mis maakonnast sa oled ? "" küsis ta .",70.0,70.0,80.0
130,71808,Dev/ilu_orlau_ud211.conllu,"Tüdruk naeratas , mängides üllatust , kuid suu...",112,118,Juudas,redundant comma,"Tüdruk naeratas , mängides üllatust , kuid suu...",90.9,90.9,95.5


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,71679,Dev/ilu_orlau_ud211.conllu,""" Kristin ! "" paanilises hirmus raputas Juudas...",0,13,""" Kristin ! """,free,paanilises hirmus raputas Juudas tüdrukut õlga...,100.0,100.0,100.0
2,71680,Train/ilu_vilep_ud211.conllu,""" Noo teate , poisid ! "" ühmas ta omaette .",12,20,", poisid",free,""" Noo teate ! "" ühmas ta omaette .",66.7,66.7,77.8
4,71682,Train/ilu_kivirahk_ud211.conllu,""" Sa oled kõigest täpselt aru saanud , õilis r...",37,51,", õilis rüütel",free,""" Sa oled kõigest täpselt aru saanud ! "" kinni...",75.0,75.0,83.3
7,71685,Train/ilu_vilep_ud211.conllu,""" Ära solvu , ema !",12,17,", ema",free,""" Ära solvu !",100.0,100.0,100.0
13,71691,Train/ilu_kivirahk_ud211.conllu,"mu põrmust lilled õitsetad , mu isamaa , mu is...",27,50,", mu isamaa , mu isamaa",free,"mu põrmust lilled õitsetad ! """,100.0,100.0,100.0
14,71692,Train/ilu_kivirahk_ud211.conllu,"mu põrmust lilled õitsetad , mu isamaa , mu is...",39,50,", mu isamaa",free,"mu põrmust lilled õitsetad , mu isamaa ! """,100.0,100.0,100.0
15,71693,Train/ilu_vilep_ud211.conllu,""" Tere , tädi Alma ! """,7,18,", tädi Alma",free,""" Tere ! """,100.0,100.0,100.0
16,71694,Train/ilu_kivirahk_ud211.conllu,""" Ole õnnistatud , vapper rüütel !",17,32,", vapper rüütel",free,""" Ole õnnistatud !",100.0,100.0,100.0
17,71695,Train/ilu_kivirahk_ud211.conllu,""" Söö , taadike , söö aga ! "" rõõmustas Petrov .",6,15,", taadike",free,""" Söö , söö aga ! "" rõõmustas Petrov .",70.0,70.0,80.0
19,71697,Train/ilu_kivirahk_ud211.conllu,"Võitlus tuleb äge , kuid Jumal hoiab oma kätt ...",87,101,", aulik rüütel",free,"Võitlus tuleb äge , kuid Jumal hoiab oma kätt ...",95.0,95.0,100.0


In [11]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,71679,Dev/ilu_orlau_ud211.conllu,""" Kristin ! "" paanilises hirmus raputas Juudas...",0,13,""" Kristin ! """,free,paanilises hirmus raputas Juudas tüdrukut õlga...,100.0,100.0,100.0
7,71685,Train/ilu_vilep_ud211.conllu,""" Ära solvu , ema !",12,17,", ema",free,""" Ära solvu !",100.0,100.0,100.0
13,71691,Train/ilu_kivirahk_ud211.conllu,"mu põrmust lilled õitsetad , mu isamaa , mu is...",27,50,", mu isamaa , mu isamaa",free,"mu põrmust lilled õitsetad ! """,100.0,100.0,100.0
14,71692,Train/ilu_kivirahk_ud211.conllu,"mu põrmust lilled õitsetad , mu isamaa , mu is...",39,50,", mu isamaa",free,"mu põrmust lilled õitsetad , mu isamaa ! """,100.0,100.0,100.0
15,71693,Train/ilu_vilep_ud211.conllu,""" Tere , tädi Alma ! """,7,18,", tädi Alma",free,""" Tere ! """,100.0,100.0,100.0
16,71694,Train/ilu_kivirahk_ud211.conllu,""" Ole õnnistatud , vapper rüütel !",17,32,", vapper rüütel",free,""" Ole õnnistatud !",100.0,100.0,100.0
21,71699,Train/aja_ee200110_osa_10_ud211.conllu,"Kas sa oled elus , armas lugeja ?",17,31,", armas lugeja",free,Kas sa oled elus ?,100.0,100.0,100.0
34,71712,Train/ilu_kivirahk_ud211.conllu,"( "" Palun , maestro , ma jätsin maiuse teile !",10,19,", maestro",free,"( "" Palun , ma jätsin maiuse teile !",100.0,100.0,100.0
41,71719,Train/ilu_kivirahk_ud211.conllu,"Su rüppe heidan unele , mu püha Eestimaa ! """,22,40,", mu püha Eestimaa",free,"Su rüppe heidan unele ! """,100.0,100.0,100.0
44,71722,Train/aja_pm19980926e_ud211.conllu,oi te poisid kuis küll põleb rind,3,12,te poisid,free,oi kuis küll põleb rind,100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [10]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
3,71681,Dev/ilu_orlau_ud211.conllu,""" Juudas ... "" sosistas ta .",0,14,""" Juudas ... """,bound,sosistas ta .,100.0,100.0,100.0
6,71684,Train/ilu_kivirahk_ud211.conllu,""" Tere hommikust , maestro , "" ütles Tõnu viis...",0,30,""" Tere hommikust , maestro , """,bound,"ütles Tõnu viisakalt , ehkki hommikust oli asi...",90.9,100.0,90.9
9,71687,Train/aja_sloleht20071217_ud211.conllu,See periood aitas kaasa minu elufilosoofia kuj...,84,90,poisid,bound,See periood aitas kaasa minu elufilosoofia kuj...,90.9,95.5,95.5
11,71689,Train/aja_ee200110_osa_7_ud211.conllu,"Tema käskija aga seisis piitsaga keset saali ,...",192,217,""" Hopp - hopp , Mieze ! """,bound,"Tema käskija aga seisis piitsaga keset saali ,...",84.4,84.4,100.0
24,71702,Train/ilu_vilep_ud211.conllu,""" Isa , "" vaatas Jan manitseval pilgul vallatl...",0,9,""" Isa , """,bound,vaatas Jan manitseval pilgul vallatlevat papsi...,73.3,73.3,86.7
43,71721,Train/aja_sloleht20071217_ud211.conllu,Tule taevas appi !,5,11,taevas,bound,Tule appi !,100.0,100.0,100.0
47,71725,Train/ilu_vilep_ud211.conllu,""" Kallis , "" püüdis ta ääriveeri juttu alustad...",2,8,Kallis,bound,""" , "" püüdis ta ääriveeri juttu alustada , "" m...",72.7,77.3,86.4
48,71726,Train/ilu_vilep_ud211.conllu,""" Siga ! "" tegi naine kokkuvõtte .",0,10,""" Siga ! """,bound,tegi naine kokkuvõtte .,100.0,100.0,100.0
54,71732,Dev/aja_ml200247_osa6_ud211.conllu,"Et kauneid , lausa lummavaid maastikke , merd ...",124,132,Inimesed,bound,"Et kauneid , lausa lummavaid maastikke , merd ...",93.5,96.8,96.8
61,71739,Dev/ilu_ruben_ud211.conllu,Tal on ligipääs minu ilusatele armastuskirjade...,76,91,", kallis tibuke",bound,Tal on ligipääs minu ilusatele armastuskirjade...,76.9,87.2,84.6


In [11]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [12]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,71678,Train/ilu_valton_ud211.conllu,""" Ära ainult valeta , noor kana ! "" ütles vaen...",22,31,noor kana,redundant comma,""" Ära ainult valeta , ! "" ütles vaenulik tädi ...",73.3,80.0,80.0
5,71683,Train/aja_luup200202_osa_1_ud211.conllu,"Kirurgid , hoidke alt !",0,8,Kirurgid,redundant comma,", hoidke alt !",100.0,100.0,100.0
8,71686,Train/ilu_kivirahk_ud211.conllu,"Jätkame otsinguid , õilsad rüütlid , ja kui me...",18,34,", õilsad rüütlid",redundant comma,"Jätkame otsinguid , ja kui me peaksime kurnatu...",100.0,100.0,100.0
10,71688,Train/ilu_indrikson_ud211.conllu,"Head reisi , "" Lufthansa ! """,13,28,""" Lufthansa ! """,redundant comma,"Head reisi ,",100.0,100.0,100.0
12,71690,Train/aja_ee200110_osa_8_ud211.conllu,""" Ütles , et poisid , pange rahad ise kokku ja...",13,19,poisid,redundant comma,""" Ütles , et , pange rahad ise kokku ja ostke ...",92.0,96.0,92.0
18,71696,Train/ilu_kivirahk_ud211.conllu,""" Kulla noorsand , meile on see tavaline asi !",2,16,Kulla noorsand,redundant comma,""" , meile on see tavaline asi !",87.5,100.0,87.5
20,71698,Train/ilu_kanep_ud211.conllu,""" Ausalt , musike , ma tõesti ei tea ! "" sõnas...",11,17,musike,redundant comma,""" Ausalt , , ma tõesti ei tea ! "" sõnas kallim...",78.6,78.6,85.7
23,71701,Dev/ilu_ruben_ud211.conllu,"Aga sekretär küsis : "" Noh , Timbu-Limbu , kud...",29,40,Timbu-Limbu,redundant comma,"Aga sekretär küsis : "" Noh , , kuda klaasipõhi...",92.9,100.0,92.9
27,71705,Train/tea_AA_05_6_ud211.conllu,"Eesti infotehnoloogid , jätkake julgelt oma na...",0,21,Eesti infotehnoloogid,redundant comma,", jätkake julgelt oma naturaalse mõistuse kasu...",90.9,90.9,90.9
28,71706,Train/ilu_kivirahk_ud211.conllu,""" Noh , taadike ! "" ütles ta kerge vene aktsen...",8,15,taadike,redundant comma,""" Noh , ! "" ütles ta kerge vene aktsendiga .",100.0,100.0,100.0


In [13]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [14]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
59,71737,Train/ilu_vilep_ud211.conllu,""" Kati ! "" pöördus ta mesimagusal häälel naise...",0,10,""" Kati ! """,unnatural,"pöördus ta mesimagusal häälel naise poole , tu...",95.8,95.8,100.0


In [15]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [16]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la


In [17]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [18]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
42,71720,Train/aja_ee200110_osa_10_ud211.conllu,UNDERILE PüHENDATUD hilisluuletuses aga kinnit...,93,98,Marie,other redundant punctuation,UNDERILE PüHENDATUD hilisluuletuses aga kinnit...,93.8,93.8,93.8


In [19]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

In [ ]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

df_un = df.loc[df['type'] == "unnatural"]
df_un

df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

df_dub = df.loc[df['type'] == "dubious"]
df_dub

df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:31:39.645597: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:31:39.977062: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:31:41.110710: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:31:41.110831: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    else:
        print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data





59it [00:06,  8.99it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")